In [24]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import EulerDiscreteScheduler # Import samplers
from diffusers import DPMSolverMultistepScheduler # Import samplers
from diffusers import DDIMScheduler # Import samplers
from diffusers.utils import load_image
from tqdm import tqdm
# Prompt generation
from prompt import prompt_generator, prompt_generator_no_human
import glob
import os

Samplers (schedulers): https://huggingface.co/docs/diffusers/api/schedulers/overview

In [2]:
print("Torch version:",torch.__version__)
print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.2.2+cu121
Is CUDA enabled? True


https://thepythoncode.com/article/control-generated-images-with-controlnet-with-huggingface#code

# First Pipeline: ControlNet + Stable Diffusion 

In [25]:
# ControlNet model
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_scribble", torch_dtype=torch.float16)

# Define stable diffusion pipeline with controlnet
# We use the realistic-vision-v20-2047 model for this example (finetunned on realistic images of people)
pipe = StableDiffusionControlNetPipeline.from_pretrained("stablediffusionapi/realistic-vision-v20-2047", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

# Enable efficient implementations using xformers for faster inference
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()

text_encoder\model.safetensors not found
Loading pipeline components...:  67%|██████▋   | 4/6 [00:09<00:06,  3.07s/it]c:\Users\aaron\Documents\git\controlnet\.myvenv\Lib\site-packages\transformers\models\clip\feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 6/6 [00:10<00:00,  1.68s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing netw

In [26]:
# Load wireframe image
image_input = load_image("wireframes\\3_4.jpg")

# Prompt
prompt = prompt_generator()
print("Prompt:",prompt)
neg_prompt = "multiple boxes in the background, cardboard, a person in the foreground, not realistic"
# Run the pipeline
image_output = pipe(prompt=prompt, negative_prompt=neg_prompt, image=image_input, num_inference_steps=20).images[0]

# Save the output
image_output.save("test.jpg")
print("Save test.jpg!")

Prompt: A long shot view of a sad man holding a colorful cardboard box with red tape from the Zara webshop in his hands, standing in front of a quite neighborhood street and cars background, the weather is rainy, high photorealistic quality.


100%|██████████| 30/30 [00:17<00:00,  1.71it/s]


Save test.jpg!


In [6]:
# Prompt generator
prompt = prompt_generator()
print(prompt)

A full shot view of a happy man holding a black cardboard box with red tape from the Amazon webshop in his hands, standing in front of a city center background, the weather is snowy, high photorealistic quality.


# Generating first synthetic dataset

"wireframes" are drawn in the bottom half region of the window, so they can easily be held  by persons.
Next, we will generate synthetic images of those wireframes, located in the directory "outputs".
- I generate random prompts
- Next I will validate them
- Finally, I will only keep the images with a single box in the image (classifier)

ControlNet model: lllyasviel/control_v11p_sd15_scribble"
Stable diffusion model: stablediffusionapi/realistic-vision-v20-2047

In [19]:
def generate_images(controlnet_model_path, sd_model_path, prompt_human):
        
    # ControlNet model
    controlnet = ControlNetModel.from_pretrained(controlnet_model_path, torch_dtype=torch.float16)

    # Define stable diffusion pipeline with controlnet
    # We use the realistic-vision-v20-2047 model for this example (finetunned on realistic images of people)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(sd_model_path, controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16)
    pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

    # Enable efficient implementations using xformers for faster inference
    pipe.enable_xformers_memory_efficient_attention()
    pipe.enable_model_cpu_offload()
    
    # Read the files in the directory
    files = os.listdir("wireframes")
    sorted_files = sorted(files, key=lambda x: (int(x.split('_')[0]), int(x.split('_')[1].split('.')[0]))) 

    for file in tqdm(sorted_files):
        image_input = load_image("wireframes\\" + file)
        prompt = prompt_generator_no_human() if prompt_human == 0 else prompt_generator()
        print(prompt)
        image_output = pipe(prompt, image_input, num_inference_steps=15).images[0]
        image_output.save(os.path.join("outputs\\", os.path.basename(file)))

In [20]:
generate_images("lllyasviel/control_v11p_sd15_scribble", "stablediffusionapi/realistic-vision-v20-2047", 1)

text_encoder\model.safetensors not found
Loading pipeline components...: 100%|██████████| 6/6 [00:04<00:00,  1.21it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
  0%|          | 0/1000 [00:00<?, ?it/s]

A low angle view of a frowning boy holding a white cardboard box with red tape from the H&M webshop in his hands, standing in front of a in the street background, the weather is cloudy, high photorealistic quality.


  0%|          | 1/1000 [00:11<3:09:21, 11.37s/it]

A medium shot view of a happy boy holding a colorful cardboard box with black tape from the Amazon webshop in his hands, standing in front of a in the street background, the weather is cloudy, high photorealistic quality.


  0%|          | 2/1000 [00:22<3:08:18, 11.32s/it]

...

100%|█████████▉| 998/1000 [3:14:05<00:21, 10.83s/it]

A low angle view of a happy woman holding a brown cardboard box with green tape from the Nike webshop in her hands, standing in front of a busy city street and people background, the weather is sunny, high photorealistic quality.


100%|█████████▉| 999/1000 [3:14:16<00:10, 10.92s/it]

A close-up view of a young boy holding a colorful cardboard box with black tape from a webshop in his hands, standing in front of a busy city street and people background, the weather is rainy, high photorealistic quality.


100%|██████████| 1000/1000 [3:14:27<00:00, 11.67s/it]
